# Weather Maps

In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import time
from scipy.stats import linregress

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# API keys
from config import api_key

In [ ]:
# these next to cells i got from the starter code to generate list of cities to use
lat_range = (-90, 90)
lng_range = (-180, 180)

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
towns = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in towns:
        towns.append(city)

# Print the city count to confirm sufficient count
len(towns)

In [ ]:
# setting my search paramaters
# towns = ['st. paul', 'albany', 'st. louis', 'miami', 'portland']
units = 'imperial'
base_url = 'http://api.openweathermap.org/data/2.5/weather?'

In [ ]:
# running thru my list of cities and grabbing the data i want
cities = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []

rec_num = 0
set_num = 0

print('Beginning Data Calls.')
print('-----------------------')

for city in towns:
    
    city_url = f"{base_url}q={city}&units={units}&appid={api_key}"
    response = requests.get(city_url)
    city_json = response.json()
    
    # sleep for one second to avoid requesting more than 60 per min
    time.sleep(1)
    
    # i want city, lat, lng, max temp, humidity, cloudiness, windspeed, country, date
    try:
        cities.append(city_json['name'])
        lat.append(city_json['coord']['lat'])
        lng.append(city_json['coord']['lon'])
        max_temp.append(city_json['main']['temp_max'])
        humidity.append(city_json['main']['humidity'])
        cloudiness.append(city_json['clouds']['all'])
        wind_speed.append(city_json['wind']['speed'])
        country.append(city_json['sys']['country'])
        date.append(city_json['dt'])
        
        if rec_num < 49:
            rec_num += 1
        else:
            rec_num = 0
            set_num +=1

        print(f'Processing Record {rec_num} of Set {set_num} | {city}')
        
    except:
        print('City not found Skipping.')
        

        
        
weather_dict = {
    'City': cities,
    'Lat': lat,
    'Lng': lng,
    'Max Temp': max_temp,
    'Humidity': humidity,
    'Cloudiness': cloudiness,
    'Wind Speed': wind_speed,
    'Country': country,
    'Date': date
}

weather_df = pd.DataFrame(weather_dict)

In [ ]:
weather_df.head()

In [ ]:
weather_df.dtypes

In [ ]:
weather_df.count()

In [ ]:
# getting rid of cities with humidity greater than 100
weather_df = weather_df.loc[weather_df['Humidity'] <= 100, :]

In [ ]:
weather_df.count()

In [ ]:
output_file = 'output/weather_data.csv'

weather_df.to_csv(output_file, index=False)

# ok, I have data.

## Scatterplots

In [ ]:
# temperature vs latitude
lat = weather_df['Lat']
temp = weather_df['Max Temp']

plt.figure(figsize=(8,6))
plt.scatter(lat, temp, color='lightgreen', edgecolor='black', alpha=.8)

plt.title('Max Temperature VS City Latitude (2/8/21)')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')

plt.grid(True)

plt.show()

In [ ]:
# humidity vs latitude
humid = weather_df['Humidity']

plt.figure(figsize=(8,6))
plt.scatter(lat, humid, color='lightgreen', edgecolor='black', alpha=.8)

plt.title('Humidity VS City Latitude (2/8/21)')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')

plt.grid(True)

plt.show()

In [ ]:
# cloudiness vs latitude
clouds = weather_df['Cloudiness']

plt.figure(figsize=(8,6))
plt.scatter(lat, clouds, color='lightgreen', edgecolor='black', alpha=.8)

plt.title('Cloudiness VS City Latitude (2/8/21)')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')

plt.grid(True)

plt.show()

In [ ]:
# wind speed VS lat
windy = weather_df['Wind Speed']

plt.figure(figsize=(8,6))
plt.scatter(lat, windy, color='lightgreen', edgecolor='black', alpha=.8)

plt.title('Wind Speed VS City Latitude (2/8/21)')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')

plt.grid(True)

plt.show()

# Regressions